In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Agents: Prompts, Schemas, and Orchestration

> In this module, we define the prompts, input/output schemas, and overall behavior of the different agents in our multi-agent system. Each agent has a specific role and expertise, allowing them to collaborate effectively to achieve complex tasks.

In [ ]:
#| default_exp agents

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pydantic import Field, BaseModel
from typing import Literal
from thucy.toolbox import *
from agents import Agent, Runner, function_tool
from thucy.configuration import *

In [ ]:
#| hide
def description(pydantic_model):
    "Print the field descriptions of a Pydantic model"
    for name, field in pydantic_model.model_fields.items():
        print(f"{name}: {field.description}\n")

We are going to first set the toolset to `seattle`. Practically, this means that our experts will subscribe to toolsets like `seattle-sql`, `seattle-schema` which you can find in the `tools.yaml` file at the root of the project.

In [ ]:
#| notest
genai_mcp.connect()
genai_mcp.set_specific_toolset('seattle')

## Expert Agent: Data Expert

The Data performs a high-level survey of available data sources without diving into detailed schemas.

### Prompt Definition:

In [ ]:
#| exports

DATA_EXPERT_PROMPT = """
# Role and Objective
You are a data expert that explores available sources. Your task is to perform a rapid, one-time survey of all accessible data sources. Your goal is to identify what databases, data stores, or connected sources exist and summarize, at a high level, what type of data each likely contains.

# Exploration Scope
Focus on high-level data assets only — databases, schemas, APIs, or data files. Do not inspect or describe tables, columns, or detailed schemas. You are creating a top-down map of the data environment, not a deep schema inventory.

# Grounding and Accuracy
Base your findings only on verified information obtained via your tools. Never infer or imagine data that was not found. If a source cannot be accessed or no metadata is available, state that fact explicitly. Keep the exploration factual, concise, and high-level.

# Instructions
- Use your tools to discover and identify all accessible data sources as efficiently as possible.
- Do not expand into schema-level details such as tables, columns, or keys.
- Keep your report short, factual, and organized — avoid long explanations or speculation.
- Do not ask the user for clarification; make reasonable assumptions and proceed.
- Prioritize speed and completeness of coverage over depth.
"""

### Input/Output
We only have output here.


#### Output

This agent returns a `DataReport` containing a concise summary of discovered data sources.

In [ ]:
#| export
#| hide

class DataReport(BaseModel):
    report: str = Field(
        ..., 
        description="A concise single-paragraph report on the available data sources."
    )

In [ ]:
description(DataReport)

report: A concise single-paragraph report on the available data sources.



In [ ]:
example_report = DataReport(
    report=("[...] A nice concise description of the environment [...]")
)

In [ ]:
example_report

DataReport(report='[...] A nice concise description of the environment [...]')

### Agent-as-Tool Wrap

Here, we expose the agent **as a tool** (i.e., `discover_data_sources`) to be used by the Lead Agent.

In [ ]:
#| export

async def discover_data_sources() -> DataReport:  # Returns a `DataReport`
    """Performs a one-time high-level discovery of all connected data sources 
    and returns a concise summary of available databases and their content domains."""
    
    tools = genai_mcp.load_toolset("schema")
    
    explorer_agent = Agent(
        name="Data Expert",
        instructions=DATA_EXPERT_PROMPT,
        model=config.experts_model,
        tools=tools,
        output_type=DataReport,
    )

    result = await Runner.run(explorer_agent, "", max_turns=config.data_expert_max_turns)

    return result.final_output

In [ ]:
#| export
discover_data_sources_tool = function_tool(discover_data_sources)

### Example

In [ ]:
#| notest

result = await discover_data_sources()

In [ ]:
#| notest

result

DataReport(report='I discovered one accessible data source: a PostgreSQL database named “Seattle” (public schema accessible). Within the public schema there is a dataset/object named “crime_data” (by name, it appears to be crime-related). No other databases, schemas, APIs, or data files were discoverable with the available tools or metadata.')

## Expert Agent: Schema Expert

The Schema Expert answers questions about database structure, relationships, and schema design.

### Prompt/Instructions

In [ ]:
#| exports

SCHEMA_EXPERT_PROMPT = """
# Role and Objective
You are a database expert specializing in answering schema-related questions for relational databases. You have tools that allow you to inspect and analyze schemas across multiple databases. Use these tools whenever they are beneficial to your analysis.

# Core Behavioral Principle
Never invent or infer schemas, tables, or columns that are not actually present in the inspected databases. If no relevant database or table exists, state that explicitly and stop. Do not describe hypothetical, example, or canonical schemas under any circumstance.

# Instructions
- Do not ask the user for clarification. For ambiguous questions, make reasonable assumptions and include them at the end of your answer—always present your answer first.
- Identify which databases or data sources are relevant to the user's question.
- Examine their schemas and explain how their structural elements (such as tables, columns, keys, and relationships) are connected.
- Respond precisely to the user's intent, providing exactly the information requested—nothing more, nothing less.
- Always state explicitly the names of the databases your answer pertains to; it is imperative that the user knows the specific databases referenced.
- Avoid speculation, assumptions, or irrelevant details.
"""

### Input/Output

#### Input

This agent expects a `SchemaQuery`.

In [ ]:
#| export
#| hide

class SchemaQuery(BaseModel):
    context_hint: str = Field(
        ..., 
        description=(
            "A high-level hint about which database or domain the tool should "
            "focus on. This helps steer the tool toward the most relevant "
            "data sources."
        )
    )
    query: str = Field(
        ..., 
        description=(
            "The natural language request or question about the schema of "
            "the available relational databases."
        )
    )

    def describe_for_agent(self) -> str:
        return (
            f"### Schema Request or Question\n{self.query}\n"
            f"### Relevant Context or Domain\n{self.context_hint}\n"
        )

In [ ]:
description(SchemaQuery)

context_hint: A high-level hint about which database or domain the tool should focus on. This helps steer the tool toward the most relevant data sources.

query: The natural language request or question about the schema of the available relational databases.



In [ ]:
schema_query_example = SchemaQuery(
    context_hint="[...] A high-level hint about which database or domain [...]",
    query="[...] The natural language request or question about the schema [...]"
)

In [ ]:
schema_query_example

SchemaQuery(context_hint='[...] A high-level hint about which database or domain [...]', query='[...] The natural language request or question about the schema [...]')

#### Output

The Schema Expert returns a `SchemaQueryAnswer` object.

In [ ]:
#| exports
#| hide

class SchemaQueryAnswer(BaseModel):
    answer: str = Field(
        ..., 
        description=(
            "The final synthesized answer to the user's schema-related question. "
            "The response must explicitly state the names of all databases it pertains to."
        )
    )

In [ ]:
description(SchemaQueryAnswer)

answer: The final synthesized answer to the user's schema-related question. The response must explicitly state the names of all databases it pertains to.



In [ ]:
schema_answer_example = SchemaQueryAnswer(
    answer="[...] The final answer to the schema-related question [...]"
)

In [ ]:
schema_answer_example

SchemaQueryAnswer(answer='[...] The final answer to the schema-related question [...]')

### Agent-as-Tool Wrap

Here, we expose the agent **as a tool** (i.e., `schema_query`) to be used by the Lead Agent.

In [ ]:
#| export

async def schema_query(query: SchemaQuery  # The `SchemaQuery` instance containing the user's schema question
                       ) -> SchemaQueryAnswer:  # The `SchemaQueryAnswer` instance containing the final answer
    """A tool that answers natural language questions about database schemas across 
    multiple potential relational database sources. It can describe tables, columns, 
    relationships, keys, and other structural details for the relevant database."""

    tools = genai_mcp.load_toolset("schema")

    agent = Agent(
        name="Schema Expert",
        instructions=SCHEMA_EXPERT_PROMPT,
        tools=tools,
        model=config.experts_model,
        output_type=SchemaQueryAnswer,
    )

    result = await Runner.run(agent, query.describe_for_agent(), max_turns=config.schema_expert_max_turns)

    return result.final_output

In [ ]:
#| export
schema_query_tool = function_tool(schema_query)

### Example

In [ ]:
#| notest


s = SchemaQuery(
    context_hint="Seattle, WA",
    query="Are there any columns related to dates of crimes?"
)

result = await schema_query(s)

In [ ]:
#| notest


result

SchemaQueryAnswer(answer='Database: Seattle\n\nTable: public.crime_data\n- offense_date — timestamp without time zone\n- report_datetime — timestamp without time zone\n\nNo other tables or columns in the Seattle database schema contain date/time data according to the inspected schema.\n\nAssumption: You meant columns that store date/time values related to crimes in the Seattle database; I inspected all tables in that database.')

In [ ]:
#| notest


print(result.answer)

Database: Seattle

Table: public.crime_data
- offense_date — timestamp without time zone
- report_datetime — timestamp without time zone

No other tables or columns in the Seattle database schema contain date/time data according to the inspected schema.

Assumption: You meant columns that store date/time values related to crimes in the Seattle database; I inspected all tables in that database.


## Expert Agent: SQL Expert

he SQL Expert translates NL questions into SQL queries and returns the evidence (in SQL) for its answers.

### Prompt/Instructions

In [ ]:
#| exports

SQL_EXPERT_PROMPT = """
# Role and Objective
You are an SQL expert focused on transparency and reproducibility. Your goal is to answer the user's question as accurately and directly as possible, while *always displaying every final SQL query* that contributed evidence to your answer. Each part of your response must clearly show the concrete SQL query (or queries) that produced the corresponding result.  You have access to tools that allow you to execute SQL queries on various databases. Use these tools whenever they are beneficial to your analysis.

# Evidence Traceability
- For each analytical statement or conclusion you present, include the exact SQL query that generated the supporting data. 
- Only show SQL queries that were successfully executed and directly used to form your final answer. 
- Do not show intermediate or failed queries. 
- When multiple queries are used (for different sub-parts of the reasoning), display each query alongside the reasoning it supports, in clearly labeled sections.

# Instructions
- Do not ask the user for clarification. For ambiguous questions, make reasonable assumptions and include them at the end of your answer — always present your answer first.
- Translate natural language questions into SQL queries, execute them, and communicate the results clearly in natural language.
- Before executing any SQL query, verify that it is well-defined and addresses a single, specific information need.
- For multi-step problems, plan the sequence of steps explicitly and execute them sequentially, integrating each intermediate result into the final coherent answer.
- Ensure each SQL query matches the SQL dialect of the target database used by the query tool.

# SQL Query Best Practices
- Plan your approach before executing any query. 
- Prefer multiple simple, well-scoped queries over single complex ones by breaking problems into logical sub-steps.
"""

### Input/Output

#### Input

This agent expects a `NLQuery` (we will later re-structure it manually in the `orchestration` module).

In [ ]:
#| export
#| hide

class NLQuery(BaseModel):
    query: str = Field(
        ..., 
        description="The user's request or question expressed in natural language."
    )
    schema_info: str = Field(
        ..., 
        description=(
            "The relevant database schema information, including the names of all "
            "databases involved, as well as details on tables, relationships, and "
            "foreign keys necessary for answering the query."
        )
    )

    def describe_for_agent(self) -> str:
        return (
            f"### NL Request or Question\n{self.query}\n"
            f"### Relevant Schema Information for the Necessary Data\n{self.schema_info}\n"
        )

In [ ]:
description(NLQuery)

query: The user's request or question expressed in natural language.

schema_info: The relevant database schema information, including the names of all databases involved, as well as details on tables, relationships, and foreign keys necessary for answering the query.



In [ ]:
nl_query_example = NLQuery(
    query="[...] The user's request or question [...]",
    schema_info="[...] The relevant database schema information [...]"
)

In [ ]:
print(nl_query_example)

query="[...] The user's request or question [...]" schema_info='[...] The relevant database schema information [...]'


Since the LLMs expect NL inputs, we must structure multi-variable schemas accordingly:

In [ ]:
print(nl_query_example.describe_for_agent())

### NL Request or Question
[...] The user's request or question [...]
### Relevant Schema Information for the Necessary Data
[...] The relevant database schema information [...]



#### Output

The SQL Expert returns a `NLQueryAnswer`. This is a common structure we will use again.

In [ ]:
#| exports
#| hide

class NLQueryAnswer(BaseModel):
    answer: str = Field(
        ..., 
        description=(
            "The final synthesized answer to the user's natural language query, expressed "
            "clearly and completely in natural language."
        )
    )

In [ ]:
description(NLQueryAnswer)

answer: The final synthesized answer to the user's natural language query, expressed clearly and completely in natural language.



In [ ]:
nl_query_answer = NLQueryAnswer(
    answer="[...] The final answer to a user's query (the 'user' can be an agent ofc) [...]"
)

In [ ]:
print(nl_query_answer)

answer="[...] The final answer to a user's query (the 'user' can be an agent ofc) [...]"


### Agent-as-Tool Wrap

Here, we expose the agent **as a tool** (i.e., `nl_query`) to be used by the Lead Agent.

In [ ]:
#| export

async def nl_query(
        query: NLQuery  # The `NLQuery` instance containing the user's question and schema info
        ) -> NLQueryAnswer:  # The `NLQueryAnswer` instance containing the final answer and SQL evidence
    """Handles natural language questions by translating them into SQL, 
    executing the queries, and returning both the results and the concrete SQL 
    statements that produced them. Each part of the answer is accompanied by the 
    exact executed SQL query that served as its evidence."""
    
    tools = genai_mcp.load_toolset("sql")
    
    agent = Agent(
        name="SQL Expert",
        instructions=SQL_EXPERT_PROMPT,
        tools=tools,
        model=config.experts_model,
        output_type=NLQueryAnswer
    )

    result = await Runner.run(agent, query.describe_for_agent(), max_turns=config.sql_expert_max_turns)

    return result.final_output

In [ ]:
#| export
nl_query_tool = function_tool(nl_query)

### Example

In [ ]:
#| notest

q = NLQuery(
    query="How many unique police sectors are there? Could you list them with counts?",
    schema_info="Seattle database, columns: beat, sector"
)

result = await nl_query(q)

In [ ]:
#| notest

print(result.answer)

Answer:

- Number of unique police sectors: 20

- List of sectors with counts (sector : count):
  U : 114,581
  E : 108,678
  K : 107,441
  B : 104,270
  M : 101,570
  D : 100,808
  R : 95,004
  Q : 91,649
  L : 87,061
  N : 85,461
  S : 76,985
  W : 75,603
  J : 72,777
  F : 72,619
  C : 66,903
  G : 66,803
  O : 47,993
  - : 11,072
  99 : 3,544
  OOJ : 176

SQL queries executed (each query below produced the stated result):

1) Query used to locate table(s) containing the columns "sector" or "beat":
SELECT table_schema, table_name, column_name
FROM information_schema.columns
WHERE column_name IN ('sector','beat')
ORDER BY table_schema, table_name;

Result returned:
  table_schema: public, table_name: crime_data, column_name: beat
  table_schema: public, table_name: crime_data, column_name: sector

2) Query to count distinct sectors:
SELECT COUNT(DISTINCT sector) AS unique_sectors
FROM public.crime_data;

Result returned:
  unique_sectors: 20

3) Query to list each sector with its cou

## Lead Agent: Verifier

The Verifier is the orchestrating agent that verifies claims by coordinating with other expert agents and grounding all conclusions in real data.

### Prompt/Instructions

In [ ]:
#| exports

VERIFIER_PROMPT = """
# Role and Objective
You are a data expert and a verifier. Your goal is to verify every claim provided to you by grounding your reasoning in real, verifiable data sources. You have access to various tools that enable you to retrieve and analyze factual data—use them whenever they enhance your analysis. You must produce a clear and structured report that summarizes your findings and **includes the exact SQL queries** that generated the supporting evidence. Your query tools are specifically designed to return these executed SQL statements for inclusion in your report.

# Data Grounding Principles
- Always begin by exploring the available data sources to understand their structure and contents before interpreting the claim. This ensures your reasoning is firmly grounded in the real data environment.
- Treat all accessible data sources as **reliable and authoritative**. You can fully trust that the data you access is accurate, complete within its scope, and suitable for verification.
- Base your conclusions strictly on what the data supports. Avoid speculation or reasoning not grounded in evidence from the data.

# Instructions
- You should always base your conclusions on real data.
- Do not ask the user for clarification. For ambiguous questions, first explore the available data environment to ground your interpretation in what the data represents. Then make reasonable assumptions about the claim's intent and clearly list them at the end of your report—after providing your answer.
- Present the collected evidence directly in your report—including any executed SQL—ensuring that each conclusion is visibly grounded in data.
- Verify each user claim by consulting available data sources.
- Examine claims thoroughly and assess whether they are supported or contradicted by the evidence.
- Use tools to obtain the information you need, delegating clear and well-scoped tasks to them when appropriate.
- For multi-step questions, plan the reasoning explicitly and execute each step through a separate tool call. Each call should address one specific information need.
- Remember: Tools are stateless. Recreate any necessary context between tool calls explicitly.
"""

### Input/Output

#### Input

This agent expects a NL `UserQuery`.

In [ ]:
#| exports
#| hide

class UserQuery(BaseModel):
    query: str

In [ ]:
user_query_example = UserQuery(
    query="[...] The user's request or question [...]"
)

In [ ]:
user_query_example

UserQuery(query="[...] The user's request or question [...]")

#### Output

The Verifier returns a `VerificationAnswer` object.

In [ ]:
#| exports
#| hide

class VerificationAnswer(BaseModel):
    report: str = Field(
        ...,
        description="The full report describing which parts of the claims are true and which are not."
    )
    verdict: Literal["Verified", "Partly Verified", "Partly Inaccurate", "Inaccurate"] = Field(
        ...,
        description=(
            "Your final verdict should be one of the following:\n"
            "- **Verified**: The overall claim is fully supported by the evidence, allowing for minor acceptable deviations (e.g., rounding, naming, or formatting differences).\n"
            "- **Partly Verified**: The overall claim is supported by the evidence, but some supporting details are incomplete, imprecise, or contain minor factual inaccuracies.\n"
            "- **Partly Inaccurate**: The overall claim contains a mixture of true and false elements, with errors substantial enough to undermine confidence in the conclusion.\n"
            "- **Inaccurate**: The overall claim is contradicted or unsupported by the evidence.\n"
        )
    )

In [ ]:
description(VerificationAnswer)

report: The full report describing which parts of the claims are true and which are not.

verdict: Your final verdict should be one of the following:
- **Verified**: The overall claim is fully supported by the evidence, allowing for minor acceptable deviations (e.g., rounding, naming, or formatting differences).
- **Partly Verified**: The overall claim is supported by the evidence, but some supporting details are incomplete, imprecise, or contain minor factual inaccuracies.
- **Partly Inaccurate**: The overall claim contains a mixture of true and false elements, with errors substantial enough to undermine confidence in the conclusion.
- **Inaccurate**: The overall claim is contradicted or unsupported by the evidence.




In [ ]:
verification_answer_example = VerificationAnswer(
    report="[...] A detailed report on the verification of the claims with concrete SQL [...]",
    verdict="Verified"
)

In [ ]:
verification_answer_example

VerificationAnswer(report='[...] A detailed report on the verification of the claims with concrete SQL [...]', verdict='Verified')

### Example

In [ ]:
#| export 

async def verify(user_query: UserQuery):  # The `UserQuery` instance containing the user's claims to verify):

    verifier_agent = Agent(
        name="Lead Verifier",
        instructions=VERIFIER_PROMPT,
        model=config.lead_model,
        tools=[nl_query_tool, schema_query_tool, discover_data_sources_tool],
        output_type=VerificationAnswer,
    )

    result = await Runner.run(verifier_agent, user_query.query, max_turns=config.lead_max_turns)

    return result.final_output

In [ ]:
#| notest

q = UserQuery(query="Seattle has less than 10 unique police sectors.")

result = await verify(q)

In [ ]:
#| notest

print(result.verdict)

Inaccurate


In [ ]:
#| notest

print(result.report)

Data sources discovered
- Seattle database (public schema). One key table relevant to police geography was identified: public.crime_data, which includes a sector column.

Schema exploration
- Table: public.crime_data — contains incident-level records with police geography fields: beat (text), precinct (text), sector (text), reporting_area (text), neighborhood (text).

Evidence from executed SQL
1) List all distinct sectors
SQL executed:
SELECT DISTINCT sector FROM public.crime_data ORDER BY sector;
Result:
- "-"
- "99"
- "B"
- "C"
- "D"
- "E"
- "F"
- "G"
- "J"
- "K"
- "L"
- "M"
- "N"
- "O"
- "OOJ"
- "Q"
- "R"
- "S"
- "U"
- "W"

2) Count distinct sectors
SQL executed:
SELECT COUNT(DISTINCT sector) AS unique_sector_count FROM public.crime_data;
Result:
- unique_sector_count: 20

Analysis
- The dataset records 20 distinct sector values (non-null), which is not less than 10.

Conclusion
- The claim that “Seattle has less than 10 unique police sectors” is contradicted by the data in the Sea

In [ ]:
#| notest

genai_mcp.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()